In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils

In [2]:
events = utils.get_events()

In [3]:
events.head(1)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


## Busco location de usuarios

In [5]:
con_city = events.loc[events.city != 'Unknown']

In [6]:
agrupadas = events.groupby(['person']).city.value_counts()

In [7]:
unstacked = agrupadas.unstack(fill_value=0)
unstacked.head(1)

city,Abadiania,Abaete,Abaetetuba,Abelardo Luz,Abrantes,Abreu e Lima,Acailandia,Acajutiba,Acarau,Acopiara,...,Woodbridge,Wrexham,Xambioa,Xanxere,Xavantina,Xaxim,Xexeu,Xinguara,Yellowknife,Ze Doca
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
feature_location = unstacked.idxmax(axis=1)
feature_location.head()

person
0008ed71                Unknown
00091926         Carlos Barbosa
00091a7a               Salvador
000ba417    Rio Grande da Serra
000c79fe         Belo Horizonte
dtype: object

In [35]:
feature_location.describe()

count         33156
unique         1941
top       São Paulo
freq           5419
dtype: object

In [38]:
person_con_location = feature_location.count()
person_totales = events.person.nunique()
print("Hay {} de {} personas con location asignado".format(person_con_location,person_totales))

Hay 33156 de 38829 personas con location asignado


### Agrego a events la info de city para todas estas personas que encontré

In [91]:
events.city = df.person.map(feature_location)

#### Selecciono los conversion por city

In [94]:
conversion = events.loc[events.event == 'conversion']

In [107]:
print("Conversiones por ciudad:")
conversion.city.value_counts().head()

Conversiones por ciudad:


São Paulo         1202
Salvador           313
Rio de Janeiro     305
Belo Horizonte     280
Brasília           130
Name: city, dtype: int64

In [108]:
print("Cantidad de usuarios únicos por ciudad:")
unique_person_per_city = events.groupby(['city']).agg({'person':'nunique'})
unique_person_per_city.sort_values(by='person',ascending=False).head()

Cantidad de usuarios únicos por ciudad:


,person
city,
São Paulo,5419
Rio de Janeiro,1763
Belo Horizonte,1230
Salvador,1156
Brasília,762


#### Agrego un feature relación de conversion/usuarios para cada location

In [153]:
test = unique_person_per_city.unstack().reset_index().drop(labels='level_0',axis=1)
test.index=test.city
test = test.drop(labels='city', axis=1)
test.columns=['users']
test.head()

,users
city,
Abadiania,1
Abaete,1
Abaetetuba,3
Abelardo Luz,2
Abrantes,2


In [167]:
conversion['city'] = conversion.index
test['conversions'] = test.index.map(conversion.city)
test.head(50)

C:\Users\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,users,conversions
city,,
Abadiania,1,NaN
Abaete,1,NaN
Abaetetuba,3,NaN
Abelardo Luz,2,NaN
Abrantes,2,NaN
Abreu e Lima,23,NaN
Acailandia,21,NaN
Acajutiba,3,NaN
Acarau,6,NaN


Index([0], dtype='object')